<a href="https://colab.research.google.com/github/Radyko/MLQuantStrategy/blob/main/AlgorithminTradingUnsupervisedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_ta

In [12]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-08-01'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df


[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-08-03 A        44.222771   47.090000   47.180000   46.900002   47.099998   
           AAL      32.271374   33.480000   33.959999   33.169998   33.470001   
           AAPL     24.263586   26.447500   26.459999   26.192499   26.202499   
           ABBV     47.158062   66.570000   67.000000   66.220001   66.809998   
           ABT      38.829514   44.950001   45.090000   44.830002   45.070000   
...                       ...         ...         ...         ...         ...   
2024-07-31 XYL     133.500000  133.500000  135.429993  132.600006  134.880005   
           YUM     132.830002  132.830002  134.029999  131.610001  132.759995   
           ZBH     111.349998  111.349998  113.080002  110.120003  111.510002   
           ZBRA    351.190002  351.190002  359.690002  348.989990  357.779999   
           ZTS     180.039993  180.039993  183.259995  179.830002  182.500000   

Price                   volume  
date       ticker               
2016-08-03 A         1508900.0  
           AAL       8254200.0  
           AAPL    120810400.0  
           ABBV      6355100.0  
           ABT       7783100.0  
...                        ...  
2024-07-31 XYL       2439500.0  
           YUM       1943100.0  
           ZBH       1592800.0  
           ZBRA       634400.0  
           ZTS       1740100.0  

[990317 rows x 6 columns]

# 2. Calculate features and technical indicators for each stock.


*   Garman-Klass Volatility
*   RSI
*   Bollinger Bands
*   ATR
*   MACD
*   Dollar Volume

Garman Klass Volatility is an approximation used to measure intraday volatility of a given asset.

Garman-Klass Volatility = (((ln(High) - ln(Low))^2)/2) -(2ln(2)-1)(ln(Adj Close) - ln(Open))^2.


In [13]:
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2 - (2*np.log(2)-1)*((np.log(df['adj close']) - np.log(df['open']))**2)
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length = 20))
pandas_ta.bbands(close=df.xs('AAPL', level=1)['adj close'], length = 20)

def calculate_bbands(x):
    bbands_result = pandas_ta.bbands(close=np.log1p(pd.Series(x)), length = 20)
    return bbands_result if bbands_result is not None else pd.DataFrame()  # Return an empty DataFrame if bbands_result is None
    #function needed, as iloc cannot work if input is None.

df['bb_lower'] = df.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:,0] if not calculate_bbands(x).empty else None)

df['bb_middle'] = df.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:,1] if not calculate_bbands(x).empty else None)

df['bb_upper'] = df.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:,2] if not calculate_bbands(x).empty else None)

def compute_atr(stock_data):
  atr = pandas_ta.atr(high=stock_data['high'], low=stock_data['low'], close = stock_data['close'], length = 14)
  return atr.sub(atr.mean()).div(atr.std())


df['atr'] = df.groupby(level = 1, group_keys = False).apply(compute_atr)

def compute_macd(close):
  macd_result = pandas_ta.macd(close=close, length = 20)
  if macd_result is not None:  # Check if macd_result is not None
    macd = macd_result.iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())
  else:
    return None  # Return None if macd_result is None

df['macd'] = df.groupby(level = 1, group_keys = False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close'] * df['volume'])/1e6
#dollar volume in millions

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-08-03 A        44.222771   47.090000   47.180000   46.900002   47.099998   
           AAL      32.271374   33.480000   33.959999   33.169998   33.470001   
           AAPL     24.263586   26.447500   26.459999   26.192499   26.202499   
           ABBV     47.158062   66.570000   67.000000   66.220001   66.809998   
           ABT      38.829514   44.950001   45.090000   44.830002   45.070000   
...                       ...         ...         ...         ...         ...   
2024-07-31 XYL     133.500000  133.500000  135.429993  132.600006  134.880005   
           YUM     132.830002  132.830002  134.029999  131.610001  132.759995   
           ZBH     111.349998  111.349998  113.080002  110.120003  111.510002   
           ZBRA    351.190002  351.190002  359.690002  348.989990  357.779999   
           ZTS     180.039993  180.039993  183.259995  179.830002  182.500000   

Price                   volume  garman_klass_vol        rsi  bb_lower  \
date       ticker                                                       
2016-08-03 A         1508900.0         -0.001517        NaN       NaN   
           AAL       8254200.0         -0.000237        NaN       NaN   
           AAPL    120810400.0         -0.002231        NaN       NaN   
           ABBV      6355100.0         -0.046807        NaN       NaN   
           ABT       7783100.0         -0.008564        NaN       NaN   
...                        ...               ...        ...       ...   
2024-07-31 XYL       2439500.0          0.000182  44.378010  4.888463   
           YUM       1943100.0          0.000166  53.203270  4.838324   
           ZBH       1592800.0          0.000351  52.383683  4.666365   
           ZBRA       634400.0          0.000322  67.599872  5.729809   
           ZTS       1740100.0          0.000107  56.187626  5.159026   

Price              bb_middle  bb_upper       atr      macd  dollar_volume  
date       ticker                                                          
2016-08-03 A             NaN       NaN       NaN       NaN      66.727739  
           AAL           NaN       NaN       NaN       NaN     266.374373  
           AAPL          NaN       NaN       NaN       NaN    2931.293535  
           ABBV          NaN       NaN       NaN       NaN     299.694200  
           ABT           NaN       NaN       NaN       NaN     302.213987  
...                      ...       ...       ...       ...            ...  
2024-07-31 XYL      4.928355  4.968247  1.892789 -0.278173     325.673250  
           YUM      4.866018  4.893712  0.721489 -0.686611     258.101977  
           ZBH      4.702034  4.737703 -0.568448  0.190412     177.358278  
           ZBRA     5.790869  5.851929  0.521713  0.873963     222.794938  
           ZTS      5.190371  5.221715  0.716857  0.688203     313.287592  

[990317 rows x 14 columns]

#3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.



*  To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.


*  Calculate 5-year rolling average of dollar volume for each stock before filtering.








In [24]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]

data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
df.unstack()[last_cols].resample('M').last().stack('ticker')], axis = 1).dropna()

data['dollar_volume'] = (data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending = False))

data = data[data['dollar_vol_rank'] < 150].drop(['dollar_vol_rank','dollar_volume'], axis = 1)

data

adj close  garman_klass_vol        rsi  bb_lower  \
date       ticker                                                      
2021-08-31 AAL      19.940001          0.000342  47.500676  2.968642   
           AAPL    149.329407         -0.000139  61.340698  4.961857   
           ABBV    107.355637         -0.005356  61.441944  4.618325   
           ABT     119.564293         -0.001141  62.864694  4.748958   
           ACN     322.019958         -0.000796  70.692928  5.710939   
...                       ...               ...        ...       ...   
2024-07-31 VZ       40.520000          0.000144  51.735733  3.687073   
           WFC      58.890114          0.000024  50.270815  4.063394   
           WMT      68.639999          0.000074  51.666269  4.241554   
           XOM     118.589996          0.000021  57.851744  4.716281   
           ZTS     180.039993          0.000107  56.187626  5.159026   

                   bb_middle  bb_upper       atr      macd  
date       ticker                                           
2021-08-31 AAL      3.040327  3.112012 -0.387914 -0.258989  
           AAPL     4.989564  5.017272  0.119032  0.448019  
           ABBV     4.658887  4.699448 -0.757495  0.396425  
           ABT      4.774887  4.800817 -0.050727  1.067105  
           ACN      5.749764  5.788590 -0.330353  1.229811  
...                      ...       ...       ...       ...  
2024-07-31 VZ       3.728617  3.770161 -0.191153 -0.152062  
           WFC      4.095216  4.127037  0.633665  0.131169  
           WMT      4.261496  4.281437  0.787756  0.648607  
           XOM      4.755448  4.794615  0.597315  0.679436  
           ZTS      5.190371  5.221715  0.716857  0.688203  

[5364 rows x 8 columns]

# 4. Calculate Monthly Returns for different time horizons as features

*   To capture time series dynamics that reflect momentum patterns, we compute historical returns using the method .pct_change(lag), which shows returns over various montlhy periods as identified by lags.



In [32]:
def calculate_returns(df):

  outlier_cutoff = 0.005 #arbitrary

  lags = [1,2,3,6,9,12]

  for lag in lags:
    df[f'return_{lag}m'] = (df['adj close']
                          .pct_change(lag)
                          .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                          .add(1)
                          .pow(1/lag)
                          .sub(1))

  return df


data = data.groupby(level = 1, group_keys= False).apply(calculate_returns).dropna()

data

adj close  garman_klass_vol        rsi  bb_lower  \
date       ticker                                                      
2022-08-31 AAL      12.990000          0.000375  40.070070  2.642446   
           AAPL    155.497101         -0.000124  45.512707  5.058516   
           ABBV    124.410988         -0.002813  33.262448  4.833324   
           ABT      98.667213         -0.000605  37.447165  4.587291   
           ACN     279.351532         -0.000674  41.112356  5.645392   
...                       ...               ...        ...       ...   
2024-07-31 VRTX    495.720001          0.000114  62.146045  6.165930   
           VZ       40.520000          0.000144  51.735733  3.687073   
           WFC      58.890114          0.000024  50.270815  4.063394   
           WMT      68.639999          0.000074  51.666269  4.241554   
           XOM     118.589996          0.000021  57.851744  4.716281   

                   bb_middle  bb_upper       atr      macd  return_1m  \
date       ticker                                                       
2022-08-31 AAL      2.736068  2.829691 -1.043338 -0.257097  -0.052516   
           AAPL     5.115718  5.172920  0.804441  0.186538  -0.031208   
           ABBV     4.868401  4.903477  0.290344 -1.610472  -0.063062   
           ABT      4.649824  4.712357  0.209162 -1.567830  -0.056872   
           ACN      5.706179  5.766965  0.746653 -0.329021  -0.058120   
...                      ...       ...       ...       ...        ...   
2024-07-31 VRTX     6.195736  6.225542  1.373761  1.324232   0.057604   
           VZ       3.728617  3.770161 -0.191153 -0.152062  -0.001379   
           WFC      4.095216  4.127037  0.633665  0.131169  -0.000842   
           WMT      4.261496  4.281437  0.787756  0.648607   0.013735   
           XOM      4.755448  4.794615  0.597315  0.679436   0.030142   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2022-08-31 AAL      0.012150  -0.100858  -0.046172  -0.033731   -0.035082  
           AAPL     0.073098   0.018897  -0.007666  -0.005097    0.003378  
           ABBV    -0.058662  -0.027083  -0.012688   0.020428    0.012363  
           ABT     -0.025846  -0.042516  -0.025163  -0.021020   -0.015881  
           ACN      0.021092  -0.010125  -0.014022  -0.022541   -0.011775  
...                      ...        ...        ...        ...         ...  
2024-07-31 VRTX     0.043399   0.080649   0.022652   0.035512    0.028860  
           VZ       0.000402   0.014092  -0.001977   0.021535    0.020210  
           WFC     -0.004852   0.002025   0.030523   0.047588    0.023656  
           WMT      0.021663   0.050365   0.038538   0.027229    0.022525  
           XOM      0.005655   0.003604   0.027048   0.015688    0.011331  

[3406 rows x 14 columns]